In [2]:
import numpy as np

import datamanager
import torch
import pathlib

device = torch.device("cuda")

config = datamanager.HyFluidNeRFDataManagerConfig(dataparser=datamanager.HyFluidDataParserConfig())
config.dataparser.data = pathlib.Path("../data/ScalarReal")
manager: datamanager.HyFluidNeRFDataManager = config.setup(device=torch.device("cuda"))
manager.to(device)

Caching all 480 images.

Output()

Caching all 120 images.

Output()

HyFluidNeRFDataManager(
  (train_ray_generator): RayGenerator()
  (eval_ray_generator): RayGenerator()
)

In [3]:
ray_bundle, batch = manager.next_train()
print(ray_bundle.shape)
print(batch['image'].shape)
print(batch['indices'].shape)

torch.Size([1024])
torch.Size([1024, 3])
torch.Size([1024, 3])


In [4]:
from nerfstudio.model_components.ray_samplers import UniformSampler
from nerfstudio.model_components.scene_colliders import NearFarCollider
from nerfstudio.cameras.rays import RaySamples

collider = NearFarCollider(near_plane=1.1, far_plane=1.5)
sampler_uniform = UniformSampler(num_samples=192)

collider(ray_bundle)
ray_samples_uniform: RaySamples = sampler_uniform(ray_bundle)

print(ray_samples_uniform.shape)

torch.Size([1024, 192])


In [5]:
start_positions = ray_samples_uniform.frustums.get_start_positions()
positions = ray_samples_uniform.frustums.get_positions()

print(start_positions.shape)
print(positions.shape)

torch.Size([1024, 192, 3])
torch.Size([1024, 192, 3])


In [6]:
np.savez_compressed(
    "positions.npz",
    start_positions=start_positions.cpu().numpy(),
    positions=positions.cpu().numpy()
)

In [7]:
image_indices = batch['indices'][:, 0]
all_frames = manager.train_dataset.metadata['all_frames']
frames = datamanager.image_idx_to_frame(image_indices, all_frames)
frames_expanded = frames.to(positions.device).view(positions.shape[0], 1, 1).expand(-1, positions.shape[1], -1)
xyzt = torch.cat((positions, frames_expanded), dim=-1)

print(all_frames)
print(image_indices)
print(frames)
print(frames.shape)
print(xyzt.shape)

[120, 120, 120, 120]
tensor([ 36.5588, 424.4061, 462.1608,  ..., 276.2364, 333.5029, 195.6115])
tensor([ 36.5588,  64.4061, 102.1608,  ...,  36.2364,  93.5029,  75.6115])
torch.Size([1024])
torch.Size([1024, 192, 4])


In [29]:
import encoder
import taichi as ti

ti.init(arch=ti.cuda)
xyzt_encoder = encoder.HashEncoderHyFluid(
    min_res=np.array([16, 16, 16, 16]),
    max_res=np.array([256, 256, 256, 128]),
    num_scales=16,
    max_params=2 ** 19,
)
xyzt_encoder.to(device)

xyzt_flat = xyzt.reshape(-1, 4)
xyzt_encoded = xyzt_encoder(xyzt_flat)

[Taichi] Starting on arch=cuda


In [30]:
print(xyzt_encoded.shape)

torch.Size([196608, 32])
cuda:0


In [47]:
from nerfstudio.field_components.mlp import MLP
import radam

mlp_base = MLP(
    in_dim=xyzt_encoder.num_scales * xyzt_encoder.features_per_level,
    num_layers=2,
    layer_width=64,
    out_dim=1,
    out_activation=torch.nn.ReLU(),
)
mlp_base.to(device)
learned_rgb = torch.nn.Parameter(torch.tensor([0.0], device=device))

grad_vars = list(mlp_base.parameters()) + [learned_rgb]
embedding_params = list(xyzt_encoder.parameters())

optimizer = radam.RAdam([
    {'params': grad_vars, 'weight_decay': 1e-6},
    {'params': embedding_params, 'eps': 1e-15}
], lr=0.01, betas=(0.9, 0.99))

In [48]:
density_flat = mlp_base(xyzt_encoded)
density = density_flat.reshape(xyzt.shape[0], xyzt.shape[1], density_flat.shape[-1])
print(f'density: {density.shape}')

density: torch.Size([1024, 192, 1])


In [49]:
raw2alpha = lambda raw, dists, act_fn=torch.nn.functional.relu: 1. - torch.exp(-act_fn(raw) * dists)
dists = ray_samples_uniform.deltas
rgb = torch.ones(3, device=device) * (0.6 + torch.tanh(learned_rgb) * 0.4)
alpha = raw2alpha(density[..., -1], dists[..., -1])
weights = alpha * torch.cumprod(torch.cat([torch.ones((alpha.shape[0], 1), device=device), 1. - alpha + 1e-10], -1), -1)[:, :-1]
rgb_map = torch.sum(weights[..., None] * rgb, -2)

print(f'dists: {dists.shape}')
print(f'alpha: {alpha.shape}')
print(f'weights: {weights.shape}')
print(f'rgb_map: {rgb_map.shape}')

dists: torch.Size([1024, 192, 1])
alpha: torch.Size([1024, 192])
weights: torch.Size([1024, 192])
rgb_map: torch.Size([1024, 3])


In [57]:
from nerfstudio.model_components.losses import MSELoss
rgb_loss = MSELoss()

image = batch['image'].to(device)
loss = rgb_loss(rgb_map, image)

optimizer.zero_grad()
loss.backward()
optimizer.step()

# print(loss.shape)


KeyboardInterrupt

